## 학습목표
 1. dataframe 제공 함수를 통한 데이터 처리 기법 숙지
 2. groupby, concat, merge, pivot, pivot_table, stack, unstack, transorm 등

### group by
  + 아래의 세 단계를 포함 (SQL의 group by 와 개념적으로는 동일, 사용법은 유사)
    - 데이터 분할
    - operation 적용
    - 데이터 병합

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 1955년부터의 유로리그 우승팀 데이터
df = pd.read_csv('./data/euro_winners.csv')

In [ ]:
df.head(15)

In [ ]:
# 국가별로 그룹핑 
nation_group = df.groupby('Nation')
type(nation_group)

In [ ]:
nation_group.groups # groups 속성으로 group 정보를 dict로 반환

In [ ]:
len(nation_group.groups) # 그룹의 개수

* group 함수
 - count, size 등

In [ ]:
nation_group.count()

In [ ]:
nation_group.size()

In [ ]:
nation_group.size().sort_values(ascending = False)

In [ ]:
# 2개 이상의 컬럼으로 grouping 가능
club_group = df.groupby(['Nation', 'Winners'])
club_group.size().sort_values(ascending = False)

* 각 리그별 경기수, 득점 데이터

In [ ]:
df = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [ ]:
df.head(20)

* inplace parameter
 - 대부분의 dataframe 연산은 원본데이터를 유지하고 복사본을 반환
 - 원본 자체를 변경하고 싶으면 inplace paramter를 True로 설정

In [ ]:
# Month column을 index로 생성
df = df.set_index('Month')

# 혹은 아래와 같이 가능
#df.set_index(['Month', 'Stat'], inplace=True)

In [ ]:
df.head()

In [ ]:
# 다시 index를 column 레벨로 변경
df = df.reset_index()

In [ ]:
df.head(100)

* groupby에 함수 사용

In [ ]:
# group by year
# 함수가 전달되는 경우 함수의 파라미터로 index 값이 전달
year_group = df.groupby(lambda month : month.split('/')[2])

In [ ]:
year_group.groups

In [ ]:
year_group.size()

In [ ]:
# index의 경우에는 column name이 아닌, 레벨로 grouping 가능
month_group = df.groupby(level = 0)

In [ ]:
# 혹은 다음과 같이 index 자체를 명시 가능
month_group = df.groupby(level = 'Month')

In [ ]:
for name, group in month_group:
    print(name, '\n')
    print(group)

### multi index grouping

* index 항목 제거

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

* 2가지 컬럼으로 인덱싱

In [ ]:
# 멀티 레벨 인덱스를 생성
df = df.set_index(['Month', 'Stat'])
df.head()

* 멀티 인덱스에서 row 선택

In [ ]:
df.loc['08/01/2012']

In [ ]:
df.loc['08/01/2012'].loc['MatchesPlayed']

In [ ]:
df.loc['08/01/2012', 'MatchesPlayed']

* 멀티 인덱스의 레벨을 이용한 그룹핑

In [ ]:
# multi level grouping
month_stat_group = df.groupby(level = [0, 1])

In [ ]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)

In [ ]:
month_stat_group = df.groupby(level = ['Month', 'Stat'])

In [ ]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)

* row 선택하기

In [ ]:
df2 = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [ ]:
df2.head()

In [ ]:
df2.loc[0] # 0은 인덱스를 의미, 여기서는 단순한 순서

In [ ]:
df2 = df2.set_index('Month')
df2.head()

In [ ]:
# index가 Month로 바뀌었으므로
df2.loc['08/01/2012']

In [ ]:
# 0번째
df2.iloc[0]

In [ ]:
df2 = df2.reset_index()

* multiple index

In [ ]:
df2 = df2.set_index(['Month', 'Stat'])
df2.head()

In [ ]:
df2.iloc[0]

In [ ]:
# multi index의 경우 ','로 구별하여 인덱싱 가능
df2.loc['08/01/2012', 'MatchesPlayed']

* grouping with multi index

In [ ]:
df2.head()

In [ ]:
stat_group = df2.groupby(level = 'Stat')
for name, group in stat_group:
    print(name)
    print(group)

In [ ]:
# group의 sum method를 이용
# 각 리그당 경기 수와 골을 나타냄
stat_group.sum()

In [ ]:
# 혹은 level을 이용하여 dataframe level에서 계산 가능
df2.sum(level = 'Stat')

In [ ]:
total_goal_df = stat_group.sum()
total_goal_df.head()

###  각 리그의 경기당 평균골 구하기

* 평균 골 Series 생성

In [ ]:
goals_per_game_df = total_goal_df.loc['GoalsScored'] / total_goal_df.loc['MatchesPlayed']
goals_per_game_df

* dataframe으로 변경

In [ ]:
pd.DataFrame(goals_per_game_df)

* Transpose하여 row, column 변경

In [ ]:
goals_per_game_df = pd.DataFrame(goals_per_game_df).T
goals_per_game_df

* mapping dictionary로 인덱스 값 변경

In [ ]:
goals_per_game_df = goals_per_game_df \
                        .rename(index = \
                                {0 : 'goals per game'})
goals_per_game_df

* 새로운 row로 추가

In [ ]:
total_goal_df.append(goals_per_game_df)

* 소수점 2자리까지 출력하도록 설정

In [ ]:
pd.options.display.float_format='{:.2f}'.format
total_goal_df.append(goals_per_game_df)

### Formating
  - https://pyformat.info/ 참고

### aggregate method
 - groupby 객체에 aggregate(집합) 함수 적용하여 그룹별 데이터 확인 가능

In [ ]:
print(stat_group)
for name, group in stat_group:
    print(name)
    print(group)

In [ ]:
stat_group.count()

In [ ]:
stat_group.aggregate(len)

In [ ]:
stat_group.aggregate(np.sum)

* multiple function 적용
  - 복수개의 column 레벨로 데이터 생성

In [ ]:
stat_group.aggregate([np.sum, np.mean, np.size])

### transform 함수

* 가상 sales data 실습
 - excel로 구성된 가상의 판매 데이터 이용하여 transformation 실습
 - excel 데이터를 읽기 위해서 xlrd 모듈이 필요
   - pip install xlrd

In [ ]:
import pandas as pd
df = pd.read_excel("data/sales_transactions.xlsx")

In [ ]:
df.head(13)

In [ ]:
df.groupby('order').sum()

In [ ]:
# ext price 컬럼만 sum
df.groupby('order')["ext price"].sum()

* 위의 정보를 다시 원래의 frame에 합치려면?
 - 아래와 같이 다시 merge 하여 합칠 수도 있음
 - 가능은 하지만, 더 쉽게 transform으로 가능

In [ ]:
order_total = df.groupby('order')["ext price"].sum().rename("total").reset_index()
order_total

In [ ]:
result = df.merge(order_total, on='order')
result["portion"] = result["ext price"] / result["total"]

In [ ]:
result

* **transform 함수 사용!**
 - group by 후 transform을 사용하면 원본의 row를 유지한 상태에서 transform에 전달된 함수를 호출하여 aggregation
 - row의 개수가 유지되므로 바로 cloumn 추가 방법을 사용하여 원본 dataframe에 추가 가능

In [ ]:
df.groupby('order')["ext price"].transform(np.sum)

In [ ]:
df['total'] = df.groupby('order')["ext price"].transform(np.sum)
df["portion"] = df["ext price"] / df["total"]
df

###  pivot 
 - dataframe의 형태를 변경
 - 변경 시, index, column, data를 명시

* ** 식물실험 데이터**
 - ctrl, tr1, tr2 환경에서의 회차별 weight 관측 데이터

In [ ]:
plant_df = pd.read_csv('data/PlantGrowth.csv')
plant_df.head()

In [ ]:
plant_df
# control(ctrl)하여 실험한 값과, 두 개의 다른 조건(tr1, tr2)을 주었을 때의 실험에 대한 결과 
# data가 stacked 됨. (말 그대로 쌓여있음)
# 데이터 분석하기 편리하지 않음

In [ ]:
# 아래와 같이 filtering하여 각각의 결과를 뽑을 수 있으나, pivot, unstack을 이용하여 분석하기 용이하도록 변형 가능
plant_df[plant_df['group'] == 'ctrl']

In [ ]:
# dataframe reshaping
# 각각 index, columns, values로 명시하여 data frame의 shape을 변경 가능
plant_df.pivot(index = 'observation', 
               columns = 'group', 
               values = 'weight')

In [ ]:
# pivot_table 으로도 동일 결과 가능
pd.pivot_table(plant_df, values = 'weight', index = 'observation', columns='group')

In [ ]:
pd.pivot_table(plant_df, values = 'weight', columns='group')

In [ ]:
pd.pivot_table(plant_df, values = 'weight', columns='observation')

* **pivot, pivot_table의 차이**
 - pivot_table 
   - pivot 결과 생성 시, 값 선택이 vague한 경우, aggfunc 사용하여 채움


In [ ]:
from collections import OrderedDict

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = pd.DataFrame(table)

In [ ]:
df

In [ ]:
df.pivot(index='Item', columns='CType', values='USD')

In [ ]:
df.pivot(index='Item', columns='CType')

In [ ]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))

df = pd.DataFrame(table)

df

In [ ]:
# Item0과 Gold에 대해서 1, 3중 어떤 것을 채워야 할지 알 수 없음
df.pivot(index='Item', columns='CType', values='USD')

In [ ]:
df.pivot_table(index='Item', 
               columns='CType', 
               values='USD',
              aggfunc = np.sum)

###  stack & unstack
 - stack : inner-most column -> inner-most index
 - unstack : inner-most index -> inner-most column
 
 - 둘은 역의 관계에 있음

In [ ]:
plant_df

In [ ]:
stacked = plant_df.set_index(['group', 'observation'])
stacked

In [ ]:
# 가장 바깥 레벨의 row -> column으로 이동
stacked.unstack()

In [ ]:
# 레벨 0의 row -> column으로 이동
stacked.unstack(0)

# 레벨의 순서가 아닌, 네임으로도 명시 가능
# stacked.unstack(level = 'group')

In [ ]:
# 컬럼을 다시 가장 하위의 row로 이동
stacked.unstack(0).stack()

### Concat

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [ ]:
df2

In [ ]:
# axis=0 기본적으로 row를 따라 합침
pd.concat([df1, df2])

In [ ]:
# axis=1로 주어 column을 따라 합치는 것도 가능
pd.concat([df1, df2], axis=1)

In [ ]:
# axis=0 기본적으로 row를 따라 합침
# 이때 기존의 인덱스를 무시하려면 ignore_index 파라미터를 True로 변경
pd.concat([df1, df2], ignore_index=True)

* column 네임이 다를 경우

In [ ]:
df3 = pd.DataFrame({'key2' : np.arange(10), 'value2' : np.random.randn(10)})

In [ ]:
# 없는 컬럼에 대해서는 nan으로 채움
pd.concat([df1, df3], ignore_index=True)

* index를 설정하고 concat

In [ ]:
df11 = df1.set_index('key1')

In [ ]:
df22 = df2.set_index('key1')

In [ ]:
pd.concat([df11, df22])

In [ ]:
result = pd.concat([df11, df22], axis=1)
result

### Merge & Join

In [ ]:
df1 = pd.DataFrame({'key1' : np.arange(8), 
                    'key2' : ['LA', 'LA', 'SF', 'SF', 'SF', 'NY', 'NY', 'LV'], 
                    'value1' : np.random.randn(8)})

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame({'key1' : [1, 1, 2, 2, 2, 3, 3], 
                    'key2' : ['LA', 'LA', 'KC', 'KC', 'NY', 'SF', 'SF'], 
                    'value1' : np.random.randn(7)})

In [ ]:
df2

* on
 - join 하고자 하는 대상이 되는 column 명시

In [ ]:
pd.merge(df1, df2, on='key1')

In [ ]:
pd.merge(df1, df2, on='key2')

* how parameter
 - SQL의 join과 같이 동작
 - inner : 기본 값, 일치 하는 값들로만 조인
 - left : left outer join과 같이 동작
 - right : right outer join과 같이 동작
 - outer : full outer join과 같이 동작, 즉 left + right

* left join
 - 왼쪽 df 기준으로 join, 없는 경우 nan 할당

In [ ]:
pd.merge(df1, df2, on='key2', how='left')

* right join
 - 오른쪽 df 기준으로 join, 없는 경우 nan 할당

In [ ]:
pd.merge(df1, df2, on='key2', how='right')

* full join
 - left join 과 right join union

In [ ]:
pd.merge(df1, df2, on='key2', how='outer')

* **multi key 조인**
 - join 하고자 할 column 리스트로 구성

In [ ]:
pd.merge(df1, df2, on=['key1', 'key2'])

In [ ]:
pd.merge(df1, df2, on=['key1', 'key2'], how='left')

In [ ]:
# 나머지 column의 이름이 겹칠 경우 suffix 지정 가능
pd.merge(df1, df2, on=['key1', 'key2'], how='left', 
         suffixes=('_left', '_right'))

* index가 있는 경우 merge
 - left_index : 왼쪽 프레임에서 인덱스를 사용할 경우 True 명시
 - right_index : 오른쪽 프레임에서 인덱스를 사용할 경우 True 명시

In [ ]:
df11 = df1.set_index('key1')
df11

In [ ]:
df22 = df2.set_index('key1')
df22

In [ ]:
# index끼리 조인할 것이라고 명시 가능
pd.merge(df11, df22, left_index=True, right_index=True)